# 🚀 Space Launch Mission Success Analysis
##  Group Presentation 4

**Team Members:**  
- Reham Abuarqoub, 9062922
- Erica Holden, 5490685
- Yu-Chen Chou (Tracy), 9006160 



#  Term Project - Space Mission Dataset Extension

## Summary of Use Case

We extended our term project by integrating a new dataset: **Global Space Mission Launches**. This data provides historical records of launch events, locations, companies, rocket types, and success status. We aim to analyze patterns and factors influencing mission outcomes. 

**Revised Hypothesis**  
**Null (H0):** The rocket cost and company do *not* significantly affect mission success.  
**Alternative (H1):** The rocket cost and company *do* significantly affect mission success.

This hypothesis update will guide our feature analysis and predictive modeling.



In [98]:
#  Imports
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector

## Load and Clean Data


In [99]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

class SpaceMissionCleaner:
    def __init__(self, filepath):
        self.df = pd.read_csv(filepath)

    def clean(self):
        df = self.df.copy()
        df.columns = df.columns.str.strip()
        df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, errors='ignore')
        df['Datum'] = pd.to_datetime(df['Datum'], errors='coerce')
        df.dropna(inplace=True)
        df['Rocket'] = pd.to_numeric(df['Rocket'], errors='coerce')
        df = df[df['Rocket'] > 0]
        df['Success'] = df['Status Mission'].apply(lambda x: 1 if 'Success' in x else 0)
        df['Company Code'] = LabelEncoder().fit_transform(df['Company Name'])
        return df

cleaner = SpaceMissionCleaner("./data/Space_Corrected.csv")
df_clean = cleaner.clean()
df_clean.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission,Success,Company Code
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07 05:12:00+00:00,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success,1,18
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06 04:01:00+00:00,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,1,2
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30 21:25:00+00:00,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success,1,17
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30 11:50:00+00:00,Atlas V 541 | Perseverance,StatusActive,145.00,Success,1,19
5,CASC,"LC-9, Taiyuan Satellite Launch Center, China",2020-07-25 03:13:00+00:00,"Long March 4B | Ziyuan-3 03, Apocalypse-10 & N...",StatusActive,64.68,Success,1,2


## Step 2: Missing Values Ratio

In [100]:
missing_ratio = df_clean.isnull().mean()
print("Missing values ratio per column:\n", missing_ratio)

Missing values ratio per column:
 Company Name      0.0
Location          0.0
Datum             0.0
Detail            0.0
Status Rocket     0.0
Rocket            0.0
Status Mission    0.0
Success           0.0
Company Code      0.0
dtype: float64


 All missing values were dropped in cleaning. Dataset is now complete.

## Step 3: Low Variance Filter

In [101]:
from sklearn.feature_selection import VarianceThreshold

class LowVariance:
    def __init__(self, df):
        self.df = df[['Rocket', 'Company Code']]

    def filter(self, threshold=0.01):
        selector = VarianceThreshold(threshold)
        reduced = selector.fit_transform(self.df)
        return pd.DataFrame(reduced, columns=self.df.columns[selector.get_support()])

lv = LowVariance(df_clean)
df_low_var = lv.filter()
df_low_var.head()

,Rocket,Company Code
0,50.00,18.0
1,29.75,2.0
2,65.00,17.0
3,145.00,19.0
4,64.68,2.0


## Step 4: High Correlation Filter

In [102]:
import numpy as np

class HighCorr:
    def __init__(self, df):
        self.df = df

    def filter(self, threshold=0.95):
        corr = self.df.corr().abs()
        upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
        return self.df.drop(columns=to_drop)
    

hc = HighCorr(df_low_var)
df_uncorrelated = hc.filter()
df_uncorrelated.head()

,Rocket,Company Code
0,50.00,18.0
1,29.75,2.0
2,65.00,17.0
3,145.00,19.0
4,64.68,2.0


## Step 5: Principal Component Analysis (PCA)

In [103]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class PCAReduction:
    def __init__(self, df):
        self.df = df

    def transform(self):
        X_scaled = StandardScaler().fit_transform(self.df)
        pca = PCA(n_components=2)
        return pd.DataFrame(pca.fit_transform(X_scaled), columns=['PC1', 'PC2'])

pca_obj = PCAReduction(df_uncorrelated)
df_pca = pca_obj.transform()
df_pca.head()

,PC1,PC2
0,0.292202,-1.085736
1,-1.413604,0.420311
2,0.265816,-0.911381
3,0.861141,-0.717538
4,-1.241319,0.592597


## Step 6: Random Forest Importance

In [104]:
from sklearn.ensemble import RandomForestClassifier

class RFImportance:
    def __init__(self, X, y):
        self.X = X.reset_index(drop=True)
        self.y = y.reset_index(drop=True)

    def rank(self):
        model = RandomForestClassifier(random_state=42)
        model.fit(self.X, self.y)
        return pd.Series(model.feature_importances_, index=self.X.columns).sort_values(ascending=False)

rf = RFImportance(df_uncorrelated, df_clean['Success'])
rf_importance = rf.rank()
print(rf_importance)

Rocket          0.648024
Company Code    0.351976
dtype: float64


## Step 7: Backward/Forward Feature Elimination/Selection
(1) Backward feature elimination

Use Random Forest + Recursive Feature Elimination (RFE) models to reduce features gradually.


In [105]:
from sklearn.feature_selection import RFE

class RFBackwardSelector:
    def __init__(self, X, y, n_features_to_select=2):
        self.X = X.reset_index(drop=True)
        self.y = y.reset_index(drop=True)
        self.n_features_to_select = n_features_to_select

    def select(self):
        model = RandomForestClassifier(random_state=42)
        # Recursive Feature Elimination
        selector = RFE(model, n_features_to_select=self.n_features_to_select, step=1)
        selector = selector.fit(self.X, self.y)
        support_mask = selector.get_support()
        selected_features = self.X.columns[support_mask]
        return selected_features.tolist()

rf_bs = RFBackwardSelector(df_uncorrelated, df_clean['Success'], n_features_to_select=1)
selected_features = rf_bs.select()
print("Selected by backward elimination:", selected_features)

Selected by backward elimination: ['Rocket']


(2) Forward feature selection

Use Random Forest + SequentialFeatureSelector models to add features gradually.

In [106]:
from sklearn.feature_selection import SequentialFeatureSelector

class RFForwardSelector:
    def __init__(self, X, y, n_features_to_select=2):
        self.X = X.reset_index(drop=True)
        self.y = y.reset_index(drop=True)
        self.n_features_to_select = n_features_to_select

    def select(self):
        # n_features = self.X.shape[1]
        # # Automatically adjust the number of features to be selected. The maximum number of features to be selected is the total number of features - 1, and the minimum number of features to be selected is 1
        # if n_features <= 1:
        #     raise ValueError("Too few features, forward selection is not possible.")
        # n_select = min(self.n_features_to_select, n_features - 1)

        model = RandomForestClassifier(random_state=42)
        selector = SequentialFeatureSelector(
            model,
            n_features_to_select=self.n_features_to_select,
            direction="forward"
        )
        selector.fit(self.X, self.y)
        support_mask = selector.get_support()
        selected_features = self.X.columns[support_mask]
        return selected_features.tolist()

rf_fs = RFForwardSelector(df_uncorrelated, df_clean['Success'], n_features_to_select=1)
selected_features = rf_fs.select()
print("Selected by forward selection:", selected_features)

Selected by forward selection: ['Company Code']


##  Hypothesis Testing Summary

### Null Hypothesis (H₀):
Rocket cost and company identity **do not significantly affect** mission success.

### Alternative Hypothesis (H₁):
Rocket cost and company identity **do significantly affect** mission success.



###  Conclusion: **Reject the Null Hypothesis**

Based on the results from our feature importance analysis:

- **Random Forest Feature Importance** showed that the `Rocket` feature had the highest influence on mission success.
- **Backward/Forward Feature Elimination/Selection** 
    - Backward elimination is `Rocket`. Most useful for predicting the Success variable (whether the launch mission was successful).
    - Forward selection is `Company Code`, because the ‘Company Code’ has the greatest contribution (highest accuracy).

These consistent findings across two independent methods provide strong evidence to **reject the null hypothesis** and **accept the alternative**:  
 *Rocket cost and company identity significantly influence mission success.*

---

###  Why This Matters:
By identifying key drivers of successful missions, organizations can prioritize spending, optimize launch conditions, and reduce risk.


We expanded our project with an enriched dataset containing global space launches. Our goal is to identify what influences mission success. After cleaning the dataset, we focused on numerical features like rocket cost and company name. Through PCA, random forest, and backward/forward feature elimination/selection, we confirmed rocket cost plays a major role in success prediction. This insight helps organizations reduce failure risk. The dataset is now clean, reduced, and ready for modeling. Our hypothesis testing and analysis provide actionable intelligence for space mission planning.
